## Install

In [0]:
!pip3 install torch torchvision numpy

## Imports

In [0]:
from matplotlib import pyplot as plt
import numpy as np
import torch as th
from torch import nn
import torchvision
from torchvision import transforms

## MNIST Dataset

In [0]:
# Download and construct MNIST dataset.
train_dataset = torchvision.datasets.MNIST(root='~/code/data/mnist/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='~/code/data/mnist/',
                                          train=False,
                                          transform=transforms.ToTensor(),
                                          download=True)

## Model

In [0]:
# Hyper-params
input_size = 28 * 28
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001


# Data loader (input pipeline)
train_loader = th.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)
test_loader = th.utils.data.DataLoader(dataset=test_dataset,
                                       batch_size=batch_size,
                                       shuffle=False)

# Model
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
    
  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

device = th.device('cuda' if th.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer.
loss_fn = nn.CrossEntropyLoss()
optimizer = th.optim.Adam(model.parameters(), lr=learning_rate)

## Train

In [10]:
num_steps = len(train_loader)
for epoch in range(num_epochs):
  for step, (images, labels) in enumerate(train_loader):
    # Forward
    images = images.reshape(-1, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = loss_fn(outputs, labels)
    
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (step + 1) % 100 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{num_steps}], '
            f'Loss: {loss.item():.4}')

Epoch [1/5], Step [100/600], Loss: 0.01871
Epoch [1/5], Step [200/600], Loss: 0.03676
Epoch [1/5], Step [300/600], Loss: 0.01147
Epoch [1/5], Step [400/600], Loss: 0.02884
Epoch [1/5], Step [500/600], Loss: 0.0222
Epoch [1/5], Step [600/600], Loss: 0.01445
Epoch [2/5], Step [100/600], Loss: 0.0148
Epoch [2/5], Step [200/600], Loss: 0.02222
Epoch [2/5], Step [300/600], Loss: 0.004325
Epoch [2/5], Step [400/600], Loss: 0.02069
Epoch [2/5], Step [500/600], Loss: 0.04607
Epoch [2/5], Step [600/600], Loss: 0.005539
Epoch [3/5], Step [100/600], Loss: 0.02551
Epoch [3/5], Step [200/600], Loss: 0.02356
Epoch [3/5], Step [300/600], Loss: 0.01658
Epoch [3/5], Step [400/600], Loss: 0.0253
Epoch [3/5], Step [500/600], Loss: 0.02541
Epoch [3/5], Step [600/600], Loss: 0.01609
Epoch [4/5], Step [100/600], Loss: 0.01872
Epoch [4/5], Step [200/600], Loss: 0.004731
Epoch [4/5], Step [300/600], Loss: 0.01147
Epoch [4/5], Step [400/600], Loss: 0.02428
Epoch [4/5], Step [500/600], Loss: 0.00266
Epoch [4/5]

## Test

In [11]:
with th.no_grad():
  correct, total = 0, 0
  for images, labels in test_loader:
    images = images.reshape(-1, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = th.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  accuracy = correct / total
  print(f'Accuracy of model on 10000 test images: {100 * accuracy:0.2f}%')

Accuracy of model on 10000 test images: 97.98%


## Save model

In [0]:
th.save(model.state_dict(), '/tmp/model.ckpt')